In [29]:
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
pd.options.display.max_colwidth = 10000
pd.options.display.max_rows = 10000

all_payments = pd.read_csv("/Users/johnclark/Documents/Finances/All_Payment_Methods020122.csv")
end_of_year = pd.read_csv("/Users/johnclark/Documents/Finances/Year End All Payment Methods020122.csv",parse_dates=[2], header=0)
end_of_year["Amount_value"] = end_of_year["Amount"].apply(lambda x: float(x[1:].replace(",","")) if x.startswith("$") else -float(x[2:].replace(",","")) if x.startswith("-$") else x)

no_credit_card = end_of_year[end_of_year["Payee"] != "WELLS FARGO CREDIT CARD"]
no_wells_credit_rec_trans = end_of_year[(end_of_year["Payee"] != "WELLS FARGO CREDIT CARD") & (~end_of_year["Description"].str.startswith("RECURRING TRANSFER")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J SAVINGS XXXXXX5751")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873"))]

utility_check = pd.read_csv("/Users/johnclark/Documents/Finances/Utility.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
utility_check["account_name"] = "utility"
umbrella_check = pd.read_csv("/Users/johnclark/Documents/Finances/Umbrella.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
umbrella_check["account_name"] = "umbrella"
savings_check = pd.read_csv("/Users/johnclark/Documents/Finances/Savings.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
savings_check["account_name"] = "savings"
primary_check = pd.read_csv("/Users/johnclark/Documents/Finances/PrimaryCheck.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
primary_check["account_name"] = "primary"
platinum_card = pd.read_csv("/Users/johnclark/Documents/Finances/PlatinumCard.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
platinum_card["account_name"] = "platinum"
cashwise_card = pd.read_csv("/Users/johnclark/Documents/Finances/CashWiseCard.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
cashwise_card["account_name"] = "cashwise"

accounts_joined = utility_check.append(umbrella_check)
accounts_joined = accounts_joined.append(savings_check)
accounts_joined = accounts_joined.append(primary_check)
accounts_joined = accounts_joined.append(platinum_card)
accounts_joined = accounts_joined.append(cashwise_card)

assert len(accounts_joined) == len(utility_check) + len(umbrella_check) + len(savings_check) + len(primary_check) + len(platinum_card) + len(cashwise_card)

accounts_joined["spending"] = accounts_joined["Amount"].apply(lambda x: x if x<0.0 else 0.0)
accounts_joined["income"] = accounts_joined["Amount"].apply(lambda x: x if x>0.0 else 0.0)

exclude_desc = {"FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK"}
tithe_desc_starts = {"RURAL INDIA MINI", "BILL PAY Church of the Open Door","CHECK # 136","BUSINESS TO BUSINESS ACH www.gracepres.co"}

accounts_joined["filtered_amount"] = accounts_joined.apply(lambda r: r["Amount"] if not r["Description"] in exclude_desc else 0.0, raw=False, axis=1)
accounts_joined["filtered_spending"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]<0.0 and not r["Description"] in exclude_desc else 0.0, raw=False, axis=1)
accounts_joined["filtered_income"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]>0.0 and not r["Description"] in exclude_desc else 0.0, raw=False, axis=1)
accounts_joined["chase"] = accounts_joined.apply(lambda r: r["Amount"] if r["Description"].startswith("CHASE") else 0.0, raw=False, axis=1)
accounts_joined["tithe"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tithe_desc_starts if r["Description"].startswith(td)]) > 0  else 0.0, raw=False, axis=1)




In [31]:
columns = ["Amount", "spending","income","filtered_amount","filtered_spending","filtered_income","chase","tithe"]
sum_table = accounts_joined.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount")[columns]
sum_table.loc["Totals"] = sum_table[columns].sum()
display(sum_table)
#display(accounts_joined["Amount"].sum())


,Amount,spending,income,filtered_amount,filtered_spending,filtered_income,chase,tithe
Date,,,,,,,,
2021-01-31 00:00:00,-3262.54,-16030.75,12768.21,-3262.54,-16030.75,12768.21,-5536.14,-2200.25
2021-02-28 00:00:00,-9186.00,-22079.10,12893.10,-9186.00,-22079.10,12893.10,-7927.58,-2200.25
2021-03-31 00:00:00,-6911.47,-39904.59,32993.12,-6911.47,-39904.59,32993.12,-5281.05,-2200.25
2021-04-30 00:00:00,66655.41,-35636.79,102292.20,16655.41,-35636.79,52292.20,-4580.39,-2200.25
2021-05-31 00:00:00,304.59,-13254.76,13559.35,304.59,-13254.76,13559.35,-3412.34,-2200.25
2021-06-30 00:00:00,-1369.97,-15697.55,14327.58,-1369.97,-15697.55,14327.58,-4841.89,-2200.25
2021-07-31 00:00:00,-6443.26,-67095.63,60652.37,-6443.26,-67095.63,60652.37,-6998.29,-7200.25
2021-08-31 00:00:00,-5549.34,-19310.10,13760.76,-5549.34,-19310.10,13760.76,-8118.73,-2200.25
2021-09-30 00:00:00,-8070.01,-21731.82,13661.81,-8070.01,-21731.82,13661.81,-5999.85,-2200.25


## Show month

In [32]:

month = 9

start_date = '2021-{}-01'.format(month)
if month < 12:
    end_date = '2021-{}-01'.format(month+1)
else:
    end_date = '2022-01-01'
view_table = accounts_joined[(accounts_joined["Date"] >= start_date) & (accounts_joined["Date"] < end_date)].sort_values("Date")
view_table["comment"] = ""

notes = {"CHECK # 132":"Arizona Department of Revenue",
         "CHECK # 131":"United States Treasury",
         "FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK":"Transfer from fidelity for tithe",
         "FRANCHISE TAX BD CASTTAXRFD 040521 XXXXX8570 CLARK III, JOHN P & HA":"Franchise tax board CASTAX"}
for idx, row in view_table.iterrows():
    if row["Description"] in notes:
        note = notes[row["Description"]]
        view_table.loc[idx,"comment"] = note
view_table



,Date,Amount,Unk,CheckNum,Description,account_name,spending,income,filtered_amount,filtered_spending,filtered_income,chase,tithe,comment
70,2021-09-01,-2091.47,*,NaN,RECURRING TRANSFER TO CLARK J REF #OP0C9RMWGL CHECKING MORTGAGE PAYMENT,primary,-2091.47,0.00,-2091.47,-2091.47,0.00,0.00,0.00,
69,2021-09-01,-114.41,*,NaN,ONLINE TRANSFER REF #IB0C9VV7BC TO WELLS FARGO CASH WISE VISA SIGNATURE CAR XXXXXXXXXXXX0560 ON 09/01/21,primary,-114.41,0.00,-114.41,-114.41,0.00,0.00,0.00,
11,2021-09-01,-400.00,*,NaN,RECURRING TRANSFER TO CLARK J REF #OP0C9S6C6Z PORTFOLIO CHECKING TO COVER GAS ELECTRIC WATER IN RENTAL,savings,-400.00,0.00,-400.00,-400.00,0.00,0.00,0.00,
16,2021-09-01,400.00,*,NaN,RECURRING TRANSFER FROM CLARK J REF #OP0C9S6C6Z SAVINGS TO COVER GAS ELECTRIC WATER IN RENTAL,umbrella,0.00,400.00,400.00,0.00,400.00,0.00,0.00,
68,2021-09-01,-133.58,*,NaN,TRANSFER TO CLARK ASTRID ON 09/01 REF #PP0C9VVV88 SEPT PHONE BILL,primary,-133.58,0.00,-133.58,-133.58,0.00,0.00,0.00,
42,2021-09-01,2091.47,*,NaN,RECURRING TRANSFER FROM CLARK J REF #OP0C9RMWGL CHECKING MORTGAGE PAYMENT,utility,0.00,2091.47,2091.47,0.00,2091.47,0.00,0.00,
41,2021-09-01,-2050.88,*,NaN,DOVENMUEHLE MTG MORTG PYMT 090121 1465323044 John P Clark III,utility,-2050.88,0.00,-2050.88,-2050.88,0.00,0.00,0.00,
67,2021-09-01,-5000.00,*,198.0,CHECK # 198,primary,-5000.00,0.00,-5000.00,-5000.00,0.00,0.00,0.00,
40,2021-09-02,-800.00,*,NaN,"RURAL INDIA MINI CONTRIBUTN 210902 XX014IOF91L5NW CLARK, JOHN",utility,-800.00,0.00,-800.00,-800.00,0.00,0.00,-800.00,
66,2021-09-03,4529.43,*,NaN,APPLE INC. PAYROLL 489652 John P Clark III,primary,0.00,4529.43,4529.43,0.00,4529.43,0.00,0.00,


In [ ]:
eoy_primary_spend = end_of_year[end_of_year["Payment Method"].str.startswith("Primary ck acct")]
primary_spend = primary_check[primary_check["Amount"] < 0]

display(eoy_primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount_value"))
display(primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount"))

